In [1]:
#Use FFTDock to dock ligands into AlphaFold models with FAD cofactor
#Inputs:
    #cofactor/pdb_with_fad/tropb.pdb
    #dock/ligands/2.pdb
    #dock/ligands/2.str #CGenFF stream file
    #toppar
#Outputs:
    #dock/grid/tropb_fftdock_3.bin
    #dock/poses/tropb_2_fftdock
    #dock/scores/tropb_2_fftdock.csv

In [2]:
import os 
import re
import pandas as pd
import numpy as np
os.environ['CHARMM_LIB_DIR'] = "/home/azamh/charmm/c48a/pycharmm_build/install/lib/"

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake
import pycharmm.grid as grid
import pycharmm.charmm_file as charmm_file
from pycharmm.select_atoms import SelectAtoms
from pycharmm.lingo import charmm_script
from pycharmm.lib import charmm as libcharmm

In [3]:
#Arguments
protein = 'tropb'
ligand = '2'
toppardir = '../../toppar'
liganddir = '../ligands'
proteindir = '../../cofactor/pdb_with_fad'
dockdir = f'../poses/{protein}_{ligand}_fftdock'
os.makedirs(dockdir, exist_ok=True)

## Read in grid box information
xcen = 57.135381
ycen = 45.1752381
zcen = 46.0617619
maxlen = 12

In [4]:
## Read in the topology and parameter file 
settings.set_bomb_level(-1)
read.rtf(os.path.join(toppardir, 'top_all36_prot.rtf'))
read.rtf(os.path.join(toppardir,'top_all36_cgenff.rtf'), append = True)
read.rtf(os.path.join(toppardir,'probes.rtf'), append = True)
read.prm(os.path.join(toppardir, 'par_all36m_prot.prm'), flex = True)
read.prm(os.path.join(toppardir, 'par_all36_cgenff.prm'), append = True, flex = True)
read.prm(os.path.join(toppardir, 'probes.prm'), append = True, flex = True)
settings.set_bomb_level(0)
charmm_script(f'stream {os.path.join(liganddir, ligand)}.str')
charmm_script(f'stream {os.path.join(toppardir, "st2_fadh.str")}')

  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::../../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name ../../toppar/top_all36_cgenff.rtf -
 CHARMM>     append
 VOPEN> Attempting to open::../../toppar/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD 

1

In [5]:
## Build protein
protein_psf = os.path.join(proteindir, f'{protein}_fad.psf')
protein_pdb = os.path.join(proteindir, f'{protein}_fad.pdb')
read.psf_card(protein_psf, append = True)
read.pdb(protein_pdb, resid = True)

  
 CHARMM>     read psf card -
 CHARMM>     name ../../cofactor/pdb_with_fad/tropb_fad.psf -
 CHARMM>     append
 VOPEN> Attempting to open::../../cofactor/pdb_with_fad/tropb_fad.psf::
 MAINIO> Protein structure file being appended from unit  91.
 psf_read_formatted: Reading PSF in the expanded format.
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:     6/22/23     21:37:41      CREATED BY USER: azamh
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        2   Number of residues   =      448
         Number of atoms         =     7040   Number of groups     =     2042
         Number of bonds         =     7130   Number of angles     =    12832
         Number of dihedrals     =    18767   Number of impropers  =     1275
         Number of cross-terms   =      447   Number of autogens   =        0
         Number of HB acceptors  =      652   Number of HB donor

In [6]:
## Generate grids
gridfile = f'../grid/{protein}_fftdock.bin'
probefile = os.path.join(toppardir, 'fftdock_c36prot_cgenff_probes.txt')
cdocker = grid.CDOCKER()
cdocker_settings = {'xCen' : xcen, 'yCen' : ycen, 'zCen' : zcen, 
            'xMax' : maxlen, 'yMax' : maxlen, 'zMax' : maxlen,
            'emax' : 2, 'maxe' : 40, 'mine' : -20, 'flag_gpu' : True, 
            'gridFile' : gridfile, 'dielec' : 3, 'flag_rdie' : True,
            'probeFile' : probefile} 
cdocker.setVar(cdocker_settings)
cdocker.generate()

 VOPEN> Attempting to open::../grid/tropb_fftdock.bin::
Use GPU to generate grids
Use default non-bond set up
  
 CHARMM>     update atom -
 CHARMM>     switch -
 CHARMM>     vswitch -
 CHARMM>     soft -
 CHARMM>     vdwe -
 CHARMM>     elee -
 CHARMM>     rdie -
 CHARMM>     cutnb 999 -
 CHARMM>     ctofnb 999 -
 CHARMM>     ctonnb 999 -
 CHARMM>     emax 2 -
 CHARMM>     mine -20 -
 CHARMM>     maxe 40 -
 CHARMM>     epsilon 3

 **** SOFT CORE AVAILABLE 
       SUGGESTED OPTIONS : RDIE SWIT VSWIT 
 FOR SPC WATER in CDIE USE : EMAX > 1000/EPS OR MINE=-100/EPS,
 FOR SPC WATER in RDIE USE : EMAX > 200/EPS

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - VDW core too soft with respect to electrostatics 
 **** INCREASE EMAX to at least       5.00

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - Elec attractive soft core starts at too high Rcut
 **** DECREASE MINE to at least     -33.33

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
  - Elec repulsive soft core starts at too high Rcut
 **** INCREAS

True

All space for grid potentials freed


In [7]:
## Prepare system
ligand_pdb = os.path.join(liganddir, f'{ligand}.pdb')
psf.delete_atoms(pycharmm.SelectAtoms().all_atoms())
read.sequence_pdb(ligand_pdb)
gen.new_segment(seg_name = "LIGA")
read.pdb(ligand_pdb, resid = True)
minimize.run_sd(nstep=1000, tolenr=1e-3, tolgrd=1e-4)



 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:        448 residues deleted.

 Message from MAPIC:          2 segments deleted.
 DELTIC:      7130 bonds deleted
 DELTIC:     12832 angles deleted
 DELTIC:     18767 dihedrals deleted
 DELTIC:      1275 improper dihedrals deleted
 DELTIC:       447 crossterm maps deleted
 DELTIC:       790 donors deleted
 DELTIC:       652 acceptors deleted
  
 CHARMM>     read sequence pdb -
 CHARMM>     name ../ligands/2.pdb
 VOPEN> Attempting to open::../ligands/2.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     1 RESIDUES
          LIG     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   1 has been generated. Its identifier is LIGA.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cle

True

In [8]:
#Get initial energy
def get_energy_df(pose_name):
    df = energy.get_energy()
    df = df[df.columns[0:10]]
    df.index = [pose_name]
    df.index.name = 'pose'
    df.columns.name = 'term'
    return df
initial_energy_df = get_energy_df('initial')
initial_energy_df


 **** SOFT CORE AVAILABLE 
       SUGGESTED OPTIONS : RDIE SWIT VSWIT 
 FOR SPC WATER in CDIE USE : EMAX > 1000/EPS OR MINE=-100/EPS,
 FOR SPC WATER in RDIE USE : EMAX > 200/EPS

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - VDW core too soft with respect to electrostatics 
 **** INCREASE EMAX to at least       5.00

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
 - Elec attractive soft core starts at too high Rcut
 **** DECREASE MINE to at least     -33.33

 ****  ERROR :POSSIBLE UNPHYSICAL RESULTS
  - Elec repulsive soft core starts at too high Rcut
 **** INCREASE MAXA to at least      66.67

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    RDIElec  SWITch   VATOm    VSWItch 
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  =999.000 CTEXNB =999.000 CTONNB =999.000 CTOFNB =999.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  3.000
 NBXMOD =      5
 VDW SOFT CORE: BEGINS AT: EMAX =      2.00 kcal/mol
 VDW SOFT CORE : LINEAR FORM 
 ELECTROSTATIC A

term,ENER,GRMS,DELTA,BOND,ANGL,UREY,DIHE,IMPR,VDW,ELEC
pose,,,,,,,,,,
initial,9.815651,1.41683,0.0,1.183762,2.072955,0.075093,6.258302,0.007516,9.622641,-9.404619


In [9]:
#Read grid 
charmm_script(f"open unit 30 read unform name {gridfile}")
charmm_script("fftg read unit 30")

  
 CHARMM>     open unit 30 read unform name ../grid/tropb_fftdock.bin
 VOPEN> Attempting to open::../grid/tropb_fftdock.bin::
 OPNLGU> Unit 30 opened for READONLY access to ../grid/tropb_fftdock.bin
  
  
 CHARMM>     fftg read unit 30
 Grid potentials read from binary file on unit  30
* EXECUTING CHARMM SCRIPT FROM PYTHON
*  DATE:     6/22/23     21:44:30      CREATED BY USER: azamh
*
 GridSetUp: Grid potentials will be set-up for  26 atom types plus electrostatics
 GridSetUp: and read from unit  30
 GridSetUp: Grid centered at  57.13538  45.17524  46.06176
 GridSetUp: Hydrogen bond grids   0.00000   0.00000   0.00000
 GridSetUp: Grid runs from (X)  51.13538 -  63.13538
 GridSetUp: Grid runs from (Y)  39.17524 -  51.17524
 GridSetUp: Grid runs from (Z)  40.06176 -  52.06176

 GridSetUp: With a grid spacing of   0.50000
 GridSetUp: Force constant at grid edge set to    300.000 kcal/mol/A^2
  


1

In [10]:
#Read list of rotations
quaternion_set = os.path.join(toppardir, 'fftdock_rotation_3.qua')
charmm_script(f"open unit 31 read form name {quaternion_set}")

  
 CHARMM>     open unit 31 read form name ../../toppar/fftdock_rotation_3.qua
 VOPEN> Attempting to open::../../toppar/fftdock_rotation_3.qua::
 OPNLGU> Unit 31 opened for READONLY access to ../../toppar/fftdock_rotation_3.qua
  


1

In [11]:
#Perform FFTDock
nsave = 500
charmm_script(f"fftg lcon ncon 1 icon 1  nrok {nsave} quau 31 sizb 100 select segid LIGA end")
charmm_script("close unit 31")

  
 CHARMM>     fftg lcon ncon 1 icon 1  nrok 500 quau 31 sizb 100 select segid LIGA end
 allocate gpu
Num of GPU Devices: 1
The device 0 is used. 
  GPU Devices Name: NVIDIA GeForce GTX 1080 Ti
  total global devices memory: 11172 MB
 flag_init_quaternions_from_file  F
FFTDOCK - num of orientations: 36864
FFTDOCK - batch_size:   100
FFTDOCK - num_batch:   369
 flag_init_lig_conformers  F
 SELRPN>     32 atoms have been selected out of     32
<FFTDock> Initializing ligand grid
Total number of grids =    27
Types of VDW grid used  =     9
BatchId / Total =     1 /   369
Batch size is 10
CuFFT result is 0
Estimated grid memory is 0
Batch size is 1000
CuFFT result is 0
Estimated grid memory is 72
BatchId / Total =     2 /   369
BatchId / Total =     3 /   369
BatchId / Total =     4 /   369
BatchId / Total =     5 /   369
BatchId / Total =     6 /   369
BatchId / Total =     7 /   369
BatchId / Total =     8 /   369
BatchId / Total =     9 /   369
BatchId / Total =    10 /   369
BatchId /

1

In [12]:
#Load grid for pose score
charmm_script(f'''
open unit 1 read unform name {gridfile}
grid read unit 1 select all end
close unit 1
''')

  
 CHARMM>     
  
 CHARMM>    open unit 1 read unform name ../grid/tropb_fftdock.bin
 VOPEN> Attempting to open::../grid/tropb_fftdock.bin::
 VCLOSE: Closing unit   30 with status "KEEP"
 OPNLGU> Unit  1 opened for READONLY access to ../grid/tropb_fftdock.bin
  
 CHARMM>    grid read unit 1 select all end
 Grid potentials read from binary file on unit   1
* EXECUTING CHARMM SCRIPT FROM PYTHON
*  DATE:     6/22/23     21:44:41      CREATED BY USER: azamh
*
 GridSetUp: Grid potentials will be set-up for  26 atom types plus electrostatics
 GridSetUp: and written to unit   1
 GridSetUp: Grid centered at  57.13538  45.17524  46.06176
 GridSetUp: Grid runs from (X)  51.13538 -  63.13538
 GridSetUp: Grid runs from (Y)  39.17524 -  51.17524
 GridSetUp: Grid runs from (Z)  40.06176 -  52.06176

 GridSetUp: With a grid spacing of   0.50000
 GridSetUp: Force constant at grid edge set to    300.000 kcal/mol/A^2
 GridSetUp: Allocating   474552 Real(chm_Real) words for grid potentials.
Hydrogen bo

1

In [13]:
#Go through each rotation
pose_energy_dfs = [initial_energy_df]
settings.set_verbosity(0)
settings.set_warn_level(-2)
for i in range(1, nsave + 1):
    
    #Set rotation
    charmm_script(f'fftg coor icon 1 irot {i}')
    
    #Calculate energy
    pose_energy_df = get_energy_df(i)
    pose_energy_dfs.append(pose_energy_df)
    
    #Save pose
    pose_pdb = os.path.join(dockdir, f'{protein}_{ligand}_{i}.crd')
    write.coor_pdb(pose_pdb)
    
settings.set_verbosity(5)
settings.set_warn_level(0)

 Coordinates of the     1th conforamtion and the     1th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     2th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     3th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     4th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     5th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     6th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     7th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     8th rotation from FFTDOCK are copied into the main coordinates.
 Coordinates of the     1th conforamtion and the     9th rotatio

-2

In [14]:
#Concat energy dataframes
energy_df = pd.concat(pose_energy_dfs).fillna(0)
energy_df

term,ENER,GRMS,DELTA,BOND,ANGL,UREY,DIHE,IMPR,VDW,ELEC
pose,,,,,,,,,,
initial,9.815651,1.41683,0.000000,1.183762,2.072955,0.075093,6.258302,0.007516,9.622641,-9.404619
1,-35.470702,1.41683,45.286353,1.183778,2.072970,0.075091,6.258301,0.007516,9.622650,-9.404632
2,-34.409033,1.41683,44.224683,1.183768,2.072976,0.075088,6.258296,0.007513,9.622628,-9.404624
3,-34.080992,1.41683,43.896642,1.183773,2.072941,0.075094,6.258299,0.007519,9.622617,-9.404595
4,-33.656231,1.41683,43.471882,1.183805,2.072953,0.075090,6.258308,0.007516,9.622594,-9.404607
...,...,...,...,...,...,...,...,...,...,...
496,-22.692962,1.41683,32.508612,1.183807,2.072960,0.075085,6.258297,0.007517,9.622595,-9.404612
497,-22.691322,1.41683,32.506972,1.183768,2.072970,0.075102,6.258321,0.007514,9.622590,-9.404612
498,-22.690904,1.41683,32.506554,1.183989,2.072908,0.075075,6.258311,0.007519,9.622472,-9.404603


In [15]:
#Get final-initial energy
delta_energy_df = energy_df.subtract(energy_df.loc['initial'].values, axis = 1)
delta_energy_df

term,ENER,GRMS,DELTA,BOND,ANGL,UREY,DIHE,IMPR,VDW,ELEC
pose,,,,,,,,,,
initial,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
1,-45.286353,0.0,45.286353,0.000015,0.000015,-0.000001,-1.628568e-06,-3.421790e-07,0.000009,-0.000013
2,-44.224683,0.0,44.224683,0.000005,0.000020,-0.000005,-6.350898e-06,-2.571214e-06,-0.000013,-0.000005
3,-43.896642,0.0,43.896642,0.000010,-0.000014,0.000002,-3.808822e-06,3.358495e-06,-0.000023,0.000023
4,-43.471882,0.0,43.471882,0.000042,-0.000002,-0.000002,5.705076e-06,-2.597988e-07,-0.000047,0.000012
...,...,...,...,...,...,...,...,...,...,...
496,-32.508612,0.0,32.508612,0.000044,0.000005,-0.000008,-4.908448e-06,7.722104e-07,-0.000046,0.000007
497,-32.506972,0.0,32.506972,0.000005,0.000015,0.000009,1.856795e-05,-2.150475e-06,-0.000050,0.000006
498,-32.506554,0.0,32.506554,0.000226,-0.000047,-0.000018,8.770946e-06,3.208228e-06,-0.000168,0.000015


In [16]:
#Save energies
scorefile = f'../scores/{protein}_{ligand}_fftdock.csv'
delta_energy_df.to_csv(scorefile)